In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import ast

In [4]:
print(pd.__version__)

2.0.2


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('data/detailed_matches.csv')

In [7]:
df.shape

(26099, 2)

In [8]:
df.head(10)

,match_id,data
0,7194183433,{'result': {'players': [{'account_id': 9170670...
1,7194159264,{'result': {'players': [{'account_id': 1470752...
2,7194108785,{'result': {'players': [{'account_id': 1402517...
3,7194103293,{'result': {'players': [{'account_id': 1525266...
4,7194092128,{'result': {'players': [{'account_id': 3550429...
5,7194076021,{'result': {'players': [{'account_id': 1545290...
6,7194068615,{'result': {'players': [{'account_id': 2362143...
7,7194041424,{'result': {'players': [{'account_id': 1425359...
8,7194026082,{'result': {'players': [{'account_id': 1734807...
9,7194014163,{'result': {'players': [{'account_id': 3127228...


In [9]:
df.isna().sum()

match_id    0
data        0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.shape

(26099, 2)

In [12]:
df['data'] = df['data'].apply(ast.literal_eval)

In [13]:
df['dt'] = df["data"].apply(lambda x: x.get("result", {}).get("start_time"))
df['dt'] = pd.to_datetime(df['dt'], unit='s')

In [14]:
df['league_id'] = df["data"].apply(lambda x: x.get("result", {}).get("league_id"))

In [61]:
df['year'] = df['dt'].dt.year
df['month'] = df['dt'].dt.month
df['day'] = df['dt'].dt.day

df = df.drop(columns = ['dt', 'match_id'])

In [62]:
all_hero_ids = set(range(1, 141))

#Create a column for every hero/team combo with a flag 0 1 if hero was picked for that team
for hero_id in all_hero_ids:
    for team_number in range(2):
        column_name = f'hero_{hero_id}_team_{team_number}'
        df[column_name] = df['data'].apply(lambda x: 1 if any(player.get('hero_id') == hero_id \
                                 and player.get('team_number') == team_number for player in x.get('result', {}).get('players', [])) else 0)

In [63]:
df['result'] = df['data'].apply(lambda x: 1 if x.get('result', {}).get('radiant_win') else 0)
df["duration"] = df["data"].apply(lambda x: x.get("result", {}).get("duration"))
df["score"] = df["data"].apply(lambda x: x.get("result", {}).get("radiant_score") +  x.get("result", {}).get("dire_score"))

In [64]:
df.groupby(['result'])['result'].count()

result
0    13592
1    12507
Name: result, dtype: int64

In [65]:
df.head(10)

,data,year,month,day,hero_1_team_0,hero_1_team_1,hero_2_team_0,hero_2_team_1,hero_3_team_0,hero_3_team_1,...,hero_137_team_1,hero_138_team_0,hero_138_team_1,hero_139_team_0,hero_139_team_1,hero_140_team_0,hero_140_team_1,result,duration,score
0,{'result': {'players': [{'account_id': 9170670...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1119,34
1,{'result': {'players': [{'account_id': 1470752...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2404,100
2,{'result': {'players': [{'account_id': 1402517...,2023,6,10,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1294,36
3,{'result': {'players': [{'account_id': 1525266...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1784,79
4,{'result': {'players': [{'account_id': 3550429...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1942,36
5,{'result': {'players': [{'account_id': 1545290...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2311,71
6,{'result': {'players': [{'account_id': 2362143...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3012,88
7,{'result': {'players': [{'account_id': 1425359...,2023,6,10,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1931,77
8,{'result': {'players': [{'account_id': 1734807...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1356,40
9,{'result': {'players': [{'account_id': 3127228...,2023,6,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1943,60


In [68]:
df = df.drop(columns = ['data'])

In [69]:
df.to_csv('data/heroes.csv', index=False)